In [5]:
import os
os.chdir('C:\\Users\\Vinim\\Documents\\github\\tcc_vinicius')

In [6]:

import pandas as pd
import numpy as np
from langchain.chains import create_tagging_chain
from langchain_openai import ChatOpenAI
import time
from tqdm import tqdm

from blob_storage import download as blob_storage 
from data_processing import util
from data_processing import llm_model

In [7]:
local_path = "C:\\Users\\Vinim\\Documents\\github\\tcc_vinicius\\data_2"

#blob_storage.download_blob_storage(local_path)

* remoção de duplicadas

In [430]:
df = util.get_dataframe(local_path)
df = df.drop_duplicates(subset=['url']).copy()
df = df.drop_duplicates(subset=['name']).copy()
df = df.drop_duplicates(subset=['descriptions']).copy()

In [431]:
# padronização dos nomes da empresa
df['key_standard'] = util.substituir_nomes(df_col= df['key'])

# tratando textos não coletados
filtro = ((df['full_text'] == 'ERROR') | (df['full_text'] ==''))
df.loc[filtro, 'full_text'] = df.loc[filtro, 'name'] 

In [ ]:
# df_march = pd.read_excel("planilhas//df_sample_march_full.xlsx")
# df_march['datePublished'].max()
# df = df[df['datePublished'] > df_march['datePublished'].max()].copy()

## Chatgpt

* llm model

In [8]:
#LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
tqdm.pandas()

### relevância

In [433]:
df['raw_classif'] = df.progress_apply(lambda x: llm_model.classify_relevance(
                                                                company=x['key_standard'],
                                                                input=f"titulo: {x['name'][:100]}\n conteúdo: {x['full_text'][:3800]}",
                                                                llm=llm), axis=1)

df['relevance'] = df['raw_classif'].apply(lambda x: int(x['relevance']) if len(list(x.keys())) > 0 else None)

100%|██████████| 5190/5190 [1:01:29<00:00,  1.41it/s]


* salvamento e leitura dos arquivos

In [441]:
# df.to_parquet('result_processing//news_classification.parquet')

: 

### Análise de sentimentos

In [4]:
df = pd.read_parquet('result_processing//news_classification.parquet')
relevance_filter = (df['relevance'] == 1)
df = df[relevance_filter].copy()

chain = llm_model.classify_sentiment_chain(llm)

In [5]:
df['raw_sentiment'] = df.progress_apply(lambda x: llm_model.classify_sentiment(chain=chain, input=f"{x['name']}\n{x['full_text']}"), axis=1)

  0%|          | 0/4227 [00:00<?, ?it/s]

100%|██████████| 4227/4227 [1:21:58<00:00,  1.16s/it]  


In [18]:
def check_sentiment_score(dictionary):
    if 'sentiment score' in dictionary:
        return dictionary['sentiment score']
    else:
        return None

In [14]:
df.loc[relevance_filter,'sentiment'] = df.loc[relevance_filter,'raw_sentiment'].apply(lambda x: x['sentiment'] if x != '' else None)
df.loc[relevance_filter,'sentiment score'] = df.loc[relevance_filter,'raw_sentiment'].apply(lambda x: check_sentiment_score(x))
# df.loc[relevance_filter,'covered topics'] = df.loc[relevance_filter,'raw_sentiment'].apply(lambda x: x['covered topics'] if x != '' else None)

In [25]:
df['sentiment score'].notna().sum()

3087

In [21]:
df['raw_sentiment'] = df['raw_sentiment'].astype('str')

In [22]:
df.to_parquet('result_processing//news_sentiment_v2.parquet')

----------------

In [9]:
df = pd.read_parquet('result_processing//news_sentiment.parquet')

In [10]:
filtro_na = df['sentiment score'].isna()

In [11]:
df.loc[filtro_na, 'name']

5       Petrobras (PETR3)(PETR4) vai investir mais de ...
6       Petrobras (PETR4): CEO diz que estatal quer di...
11      Petrobras perde recurso no Carf e não consegue...
13      Petrobras não quer comprar fatia da Braskem, m...
16      Petrobras quer dividir gestão da Braskem com n...
                              ...                        
9882    Guide troca três ações em carteira recomendada...
9883    Comparar com Companhia Paranaense de Energia (...
9910    BTG Pactual: GIC Private Limited passa a deter...
9919    Santander libera 219 casas para leilão em 17 e...
9920    Banco realiza leilão de mais de 200 imóveis co...
Name: name, Length: 1140, dtype: object

In [12]:
chain = llm_model.classify_sentiment_chain(llm)

In [13]:
df.loc[filtro_na,'raw_sentiment'] = df[filtro_na].progress_apply(lambda x: llm_model.classify_sentiment(chain=chain, input=f"{x['name']}\n{x['full_text']}"), axis=1)

  0%|          | 0/1140 [00:00<?, ?it/s]

  0%|          | 2/1140 [00:00<04:35,  4.13it/s]

{'input': 'Petrobras (PETR3)(PETR4) vai investir mais de US$ 1,5 bi para reduzir emissões\nde CO2 no Mero\n\n\nPetrobras (PETR3)(PETR4) vai investir mais de US$ 1,5 bi para reduzir emissões\nde CO2 no Mero\n\n', 'text': {}}


  0%|          | 3/1140 [00:01<08:24,  2.26it/s]

{'input': 'Petrobras (PETR4): CEO diz que estatal quer dividir gestão na Braskem (BRKM5)\n\n\nPetrobras (PETR4): CEO diz que estatal quer dividir gestão na Braskem (BRKM5)\n\n', 'text': {}}


...

{'input': 'BTG Pactual: GIC Private Limited passa a deter 5,644% do total das ações\npreferenciais da companhia\n\n\nBTG Pactual: GIC Private Limited passa a deter 5,644% do total das ações\npreferenciais da companhia\n\n', 'text': {}}


100%|██████████| 1140/1140 [30:22<00:00,  1.44it/s]

{'input': 'Santander libera 219 casas para leilão em 17 estados\n\n\nO Banco Santander anuncia a realização de um grande leilão de imóveis, marcando uma oportunidade única para adquirir propriedades residenciais, comerciais, além de lotes e terrenos em várias localidades do Brasil. Este evento, organizado em colaboração com a Biasi Leilões, destaca-se por sua abrangência nacional, abarcando uma vasta gama de imóveis distribuídos por diversos estados.\n\nSantander libera 219 casas para leilão em 17 estados | Foto: Jeane de Oliveira / Noticiadamanha.com.br\n\nDetalhes do evento e como participar\n\nO leilão ocorrerá no dia 15 de abril, com início às 15h, horário de Brasília, e será realizado inteiramente online através do site da Biasi Leilões. Eduardo Consentino será o responsável por conduzir o evento, que promete ser um marco no calendário de leilões do país.\n\nSerão ofertados ao todo 219 imóveis, entre eles apartamentos, casas, terrenos e propriedades comerciais, refletindo a divers

100%|██████████| 1140/1140 [30:22<00:00,  1.60s/it]

{'input': 'Banco realiza leilão de mais de 200 imóveis com lances entre R$ 57 mil e R$ 2\nmilhões\n\n\nBanco realiza leilão de mais de 200 imóveis com lances entre R$ 57 mil e R$ 2 milhões\n\nSão casas, apartamentos, lotes e imóveis comerciais em 239 lotes, disponíveis em 17 estados do país', 'text': {}}


In [19]:
# df.loc[filtro_na,'sentiment'] = df.loc[filtro_na,'raw_sentiment'].apply(lambda x: x['sentiment'] if x != '' else None)
df.loc[filtro_na,'sentiment score'] = df.loc[filtro_na,'raw_sentiment'].apply(lambda x: check_sentiment_score(x))
# df.loc[relevance_filter,'covered topics'] = df.loc[relevance_filter,'raw_sentiment'].apply(lambda x: x['covered topics'] if x != '' else None)